# JupyterHubのパラメータ変更

---

構築済みのJupyterhubのパラメータを再設定します。  
各パラメータは、未指定の場合、既存の設定から変更しません。

## 事前準備

### UnitGroup名

JupyterHubの構築環境となるVCPのUnitGroup名を指定します。

グループ名を確認するために `group_vars`ファイル名の一覧を表示します。

In [ ]:
!ls -1 --hide all group_vars/

上のセルの出力結果を参考にして、UnitGroup名を次のセルに指定してください。

In [ ]:
# (例)
# ugroup_name = 'JupyterHub'

ugroup_name = 

### チェック

対象となるVCノードがAnsibleによって操作できることを確認します。

Ansibleの設定ファイルの場所を環境変数に設定しておきます。

In [ ]:
from pathlib import Path
import os

cfg_ansible = Path('ansible.cfg')
if cfg_ansible.exists():
    os.environ['ANSIBLE_CONFIG'] = str(cfg_ansible.resolve())

構築対象となるVCノードにアクセスできることを確認します。

In [ ]:
target_hub = f'{ugroup_name}_manager'

!ansible {target_hub} -m ping

UnitGroup名に対応する group_varsファイルが存在していることを確認します。

In [ ]:
!test -f group_vars/{ugroup_name}

JupyterHubのサービスが実行されていることを確認します。

In [ ]:
!ansible {target_hub} -a 'docker service ps {ugroup_name}_jupyterhub'

group_varsから現在の設定値を読み込みます。

In [ ]:
import copy
import yaml
from pathlib import Path

gvars_path = Path(f'group_vars/{ugroup_name}')
with gvars_path.open() as f:
    gvars = yaml.safe_load(f)

gvars_old = copy.deepcopy(gvars)

## パラメータの設定

JupyterHubシステムに関する各種設定項目を指定します。

ここで設定された項目はansible変数として保存され、ファイルのmanagerノードへの配置時に各ファイルに反映されます。

### 必須設定項目

#### パラメータ類

In [ ]:
# JupyterHubのホスト名(FQDN) (例) 'www.sample.org'
jupyterhub_fqdn = 

In [ ]:
# lti1.3認証連携情報(moodle等)
# lms プラットフォームID (例) 'www.sample.org'
lms_platform_id = 
# lms クライアントID 
lms_cliend_id = 

受講生一覧取得方法を設定します。  
Moodleのバージョンが4.0.0以上であれば、以下のセルを変更する必要はありません。  
Moodleのバージョンが4.0.0未満で、NRPS（Names and Role Provisioning Services）を利用できない場合に、moodleのwebserviceを利用します。（https://tracker.moodle.org/browse/MDL-75279）
その場合、`get_course_member_method`には`'moodle_api'`を指定し、Moodleで発行したトークンを`lms_api_token`に指定してください。

In [ ]:
# get_course_member_method = 'moodle_api'
get_course_member_method = ''
lms_api_token = ''

### 任意設定項目

#### パラメータ類

jupyterhubで使用するDB等の接続情報等を設定します。

In [ ]:
# メールアドレスドメイン設定
email_domain = 

In [ ]:
# JupyterhubDB接続情報
db_user = 
db_password = 

In [ ]:
# Jupyterhub用LDAPサーバ（ローカルLDAP）
ldap_admin = 
ldap_password = 

In [ ]:
# Jupyterhub 初期設定
jupyterhub_admin_users = 

In [ ]:
# single-user notebook server コンテナイメージ
singleuser_image = 

In [ ]:
# 共有ディレクトリパス
home_directory_root = 
share_directory_root = 

In [ ]:
# dockerネットワーク
swarm_network = 

In [ ]:
# ユーザを一意に識別するキー
lti_username_key = 

In [ ]:
# ユーザのcookieの有効日数(0.25日=6時間)
cookie_max_age_days = 

In [ ]:
# Jupyterhubコンテナイメージ
jupyterhub_image = 

In [ ]:
# single-user notebook server コンテナをDocker Swarmのどのノードで起動するか
# 「011-VCノード作成」で、worker_nodes(workerノード数)に0を指定した場合、"manager"を指定します。
# node_role = 'manager'
node_role = 

In [ ]:
# デフォルトビュー
# "/tree" を指定すると、デフォルトのUIがNotebookになります。
# "/lab" を指定すると、デフォルトのUIがlabになります。
default_url = 

single-user notebook serverコンテナのリソース制限等を設定します。  
変更する場合、以下の内容を書き換えてください。  

[参考: DockerSpawner API](https://jupyterhub-dockerspawner.readthedocs.io/en/latest/api/index.html)

また、idle状態のsingle-user notebook serverコンテナを定期的にシャットダウンするサービスを稼働させるため、idle状態でのタイムアウトの時間設定を変更する場合もこちらで行ってください。デフォルトでは、1分に1回、idle状態かどうかのチェックを行い、600秒＝10分間idle状態が続いているコンテナはシャットダウンするように設定しています。  
idle状態のコンテナも起動したままにしたい場合、`cull_server_idle_timeout`の値を0に設定してください。  

[参考: jupyterhub-idle-culler](https://github.com/jupyterhub/jupyterhub-idle-culler)

In [ ]:
import copy
import yaml
from pathlib import Path

path = Path('edit/jupyterhub_params.yaml')
with path.open() as f:
    params = yaml.safe_load(f)
params_old = copy.deepcopy(params)

params.update({
    'resource': {
        'groups': {
            'student': {
                'mem_limit': '1G',
                'cpu_limit': 0.5,
                'mem_guarantee': 0,
                'cpu_guarantee': 0,
            },
            'teacher': {
                'mem_limit': '1G',
                'cpu_limit': 1.0,
                'mem_guarantee': 0,
                'cpu_guarantee': 0,
            },
        },
    },
    'cookie_max_age_days': 0.25,
    'cull_server': {
        'cull_server_idle_timeout': 600,
        'cull_server_every': 60,
        'cull_server_max_age': 0,
    },
})

変更前後の差分を表示します。  
意図した変更がされているかを確認してください。

In [ ]:
%run scripts/nb_utils.py
print(get_diff(params_old, params))

問題なければ、変更後の内容を保存します。

In [ ]:
with path.open(mode='w') as f:
    yaml.safe_dump(params, stream=f)

#### その他Jupyterhubで設定可能な項目

その他、Jupyterhubで設定可能な項目は、[Jupyterhub公式ドキュメント](https://jupyterhub.readthedocs.io/en/1.4.2/reference/config-reference.html)等を参照し、`jupyterhub_config.py`を直接編集してください。

以下のセルを実行すると、編集対象の`jupyterhub_config.py`の編集画面へのリンクを表示します。

In [ ]:
import os
from pathlib import Path
conf = Path('edit').absolute() / 'jupyterhub_config.py'
generate_edit_link(conf)

### オーバーレイネットワークの指定

JupyterHubのコンテナはDocker Swarmのオーバーレイネットワークで実行されます。オーバーレイネットワークに割り当てるサブネットを指定します。

> 基本的にはデフォルト値からの変更は不要ですが、VCノードに割り当てられているIPアドレスと範囲が重複している場合は他のサブネットに変更して下さい。

In [ ]:
# (例)
# jupyterhub_backend = '10.1.0.0/20'

jupyterhub_backend = 

指定されたサブネットがVCノードに割り当てられているIPアドレスと重なっていないことをチェックします。次のセルを実行してエラーとならないことを確認してください。

In [ ]:
!ansible-playbook -v -e jupyterhub_backend={jupyterhub_backend} -l {ugroup_name} \
    playbooks/check-subnet.yml

### パラメータの確認

変更前後の差分を表示します。  
**意図した内容になっている・意図していない変更がされていないことを必ず確認してください**

In [ ]:
global_vars = globals()
for key, value in gvars_old.items():
    if key in global_vars:
        gvars[key] = global_vars[key]

In [ ]:
%run scripts/nb_utils.py
print(get_diff(gvars_old, gvars))

### パラメータの保存

ここまで指定したパラメータを ansible の変数として `group_vars`ファイルに保存します。

In [ ]:
import yaml
from pathlib import Path

gvars_path = Path(f'group_vars/{ugroup_name}')

if gvars['worker_nodes'] == 0:
    node_role = 'manager'

with gvars_path.open(mode='w') as f:
    yaml.safe_dump(gvars, stream=f)

## コンテナの起動

### docker-compose.yml の配置

複数のコンテナに関する設定をまとめて扱うために `docker-compose.yml` を利用します。

ここまでに指定されたパラメータに応じた`docker-compose.yml`, `default.conf`を生成し、構築環境に配置します。

まずは、チェックモードで確認を行います。

In [ ]:
!ansible {target_hub} -CDv -m template \
    -a 'src=template/jupyterhub/docker-compose.yml dest={{{{base_dir}}}}/jupyterhub backup=yes'
!ansible {target_hub} -CDv -m template \
    -a 'src=template/jupyterhub/nginx/default.conf dest={{{{base_dir}}}}/jupyterhub/nginx backup=yes'
!ansible {target_hub} -CDv -m synchronize \
    -a 'src=edit/lms_web_service.py dest={{{{jupyterhub_dir}}}}/jupyterhub'
!ansible {target_hub} -CDv -m synchronize \
    -a 'src=edit/jupyterhub_config.py dest={{{{jupyterhub_dir}}}}/jupyterhub'
!ansible {target_hub} -CDv -m synchronize \
    -a 'src=edit/jupyterhub_params.yaml dest={{{{jupyterhub_dir}}}}/jupyterhub'

実際に各ファイルの配置を行います。

In [ ]:
!ansible {target_hub} -Dv -m template \
    -a 'src=template/jupyterhub/docker-compose.yml dest={{{{base_dir}}}}/jupyterhub backup=yes'
!ansible {target_hub} -Dv -m template \
    -a 'src=template/jupyterhub/nginx/default.conf dest={{{{base_dir}}}}/jupyterhub/nginx backup=yes'
!ansible {target_hub} -Dv -m synchronize \
    -a 'src=edit/lms_web_service.py dest={{{{jupyterhub_dir}}}}/jupyterhub'
!ansible {target_hub} -Dv -m synchronize \
    -a 'src=edit/jupyterhub_config.py dest={{{{jupyterhub_dir}}}}/jupyterhub'
!ansible {target_hub} -Dv -m synchronize \
    -a 'src=edit/jupyterhub_params.yaml dest={{{{jupyterhub_dir}}}}/jupyterhub'

### コンテナの起動

コンテナを起動します。

In [ ]:
!ansible {target_hub} -a 'chdir={{{{base_dir}}}}/jupyterhub \
    docker stack deploy -c docker-compose.yml {{{{ugroup_name}}}}'

リソース制限定義をリロードするため、jupyterhubサービスを更新します。

In [ ]:
!ansible {target_hub} -a 'docker service update {{{{ugroup_name}}}}_jupyterhub --force'

コンテナの起動状態を確認します。

In [ ]:
!ansible {target_hub} -a 'docker stack ps {{{{ugroup_name}}}}'

Masterノードにて、jupyterhubのヘルスチェック用URLに向けて、curlコマンドを実行してみます。    
起動に時間がかかる場合があるため、指定回数を上限とし、成功するまでリトライします。  
ステータスコード`200`が返ると、このセルは正常終了します。  

※managerノード（のVM）へのアクセス制限を行っている場合、この構築環境からはアクセスできない場合があります。その場合は、許可されたアクセス元から、https://{master_fqdn}/hub/healthにGETリクエストを行うか、https://{master_fqdn}/にブラウザでアクセスし、画面が表示されれば問題ありません。
このようなアクセス制限を行う場合は、Jupyterhubに設定しているグローバルIPアドレスからMoodleへのアクセスを許可する必要があります。

In [ ]:
import time

# 規定回数
retry_max = 18
err = None

for retry in range(retry_max):
    try:
        !ansible {target_hub} -m uri -a "url=https://{jupyterhub_fqdn}/hub/health"
        break

    except Exception as e:
        print("retry")
        err = e
        time.sleep(10)
else:
    raise err